# Advent of Code 2021 - Day 16

In [2]:
EXAMPLE = 0
filename = "input.txt"
if EXAMPLE: filename = "example.txt"
    
data = []
with open(filename, "r") as f:
    for line in f:
        data += [line.strip()]
        
data[:5]

['220D4B80491FE6FBDCDA61F23F1D9B763004A7C128012F9DA88CE27B000B30F4804D49CD515380352100763DC5E8EC000844338B10B667A1E60094B7BE8D600ACE774DF39DD364979F67A9AC0D1802B2A41401354F6BF1DC0627B15EC5CCC01694F5BABFC00964E93C95CF080263F0046741A740A76B704300824926693274BE7CC880267D00464852484A5F74520005D65A1EAD2334A700BA4EA41256E4BBBD8DC0999FC3A97286C20164B4FF14A93FD2947494E683E752E49B2737DF7C4080181973496509A5B9A8D37B7C300434016920D9EAEF16AEC0A4AB7DF5B1C01C933B9AAF19E1818027A00A80021F1FA0E43400043E174638572B984B066401D3E802735A4A9ECE371789685AB3E0E800725333EFFBB4B8D131A9F39ED413A1720058F339EE32052D48EC4E5EC3A6006CC2B4BE6FF3F40017A0E4D522226009CA676A7600980021F1921446700042A23C368B713CC015E007324A38DF30BB30533D001200F3E7AC33A00A4F73149558E7B98A4AACC402660803D1EA1045C1006E2CC668EC200F4568A5104802B7D004A53819327531FE607E118803B260F371D02CAEA3486050004EE3006A1E463858600F46D8531E08010987B1BE251002013445345C600B4F67617400D14F61867B39AA38018F8C05E430163C6004980126005B801CC0417080106005000CB4002D7A801AA006

# Part 1

In [3]:
def add_leading_zeros_to_make_multiple_of_four(in_b):    
    if len(in_b) % 4 != 0:        
        new_len = len(in_b)
        while(True):
            new_len += 1
            if new_len % 4 == 0:
                break
        new = ('0' * (new_len - len(in_b))) + in_b
        return new
    
    else:
        return in_b
    
add_leading_zeros_to_make_multiple_of_four('111000000000000110111101000101001010010001001000000000')

'00111000000000000110111101000101001010010001001000000000'

In [4]:
class Packet_Node:
    def __init__(self):
        self.children = []
        self.version = None
        self.type_id = None
        self.begin = 0
        self.literal = None
        self.end = None
        
    def Print_Tree(self, depth = 0):
        print('\t' * depth, 'Node:', (self.begin, self.end), 'Children ({})'.format(len(self.children)), 'Type ID:', self.type_id, 'Version:', self.version, 'Literal:', self.literal)
        for x in self.children:
            x.Print_Tree(depth + 1)
        
        
Packet_Node().Print_Tree()

 Node: (0, None) Children (0) Type ID: None Version: None Literal: None


In [5]:
def Create_Tree_From_Packets(in_b, parent_node, begin, num_children_packets = None):
    print('\n\npackets_parse, packet:', in_b, 'begin', begin, 'num_children_packets:', num_children_packets)
    
    if num_children_packets == 0:
        print('Returning')
        return
    
    children_rendered = 0

    print('Create Node')
    node = Packet_Node()
    parent_node.children += [node]
    
    version_b = in_b[:3]
    type_id_b = in_b[3:6]
    version = int(version_b, 2)
    type_id = int(type_id_b, 2)
    
    node.version = version
    node.type_id = type_id
    
    print('version', version_b, '=', version)
    print('type_id', type_id_b, '=', type_id)
    
    if type_id == 4:
        print('Literal Packet')
        
        literal_b = ''
        number = in_b[6:]
        done_mark = 0
        for i in range(0, len(number), 5):
            chunk = number[i: i + 5]
            done_mark += 5
            if chunk[0] == '1':
                literal_b += chunk[1:]
            elif chunk[0] == '0':
                literal_b += chunk[1:]
                break
        
        node.literal = int(literal_b, 2)
        print('Literal:', literal_b, '=', node.literal)
        
        node.begin = begin
        node.end = begin + 6 + done_mark - 1
        print('Node markers:', node.begin, node.end)
        
        remaining = in_b[6 + done_mark : ]
        
        if len(remaining) and remaining != len(remaining) * '0':
            if num_children_packets:
                Create_Tree_From_Packets(remaining, parent_node, node.end + 1, num_children_packets - 1)
            else:
                Create_Tree_From_Packets(remaining, parent_node, node.end + 1)
        else:
            print('No remaining!', remaining)
            parent_node.end = begin + 6 + done_mark - 1
                
    else:
        length_type_ID = in_b[6]
        print('length_type_ID:', length_type_ID)
        
        if length_type_ID == '0':
            
            total_length_b = in_b[7 : 7 + 15]
            total_length = int(total_length_b, 2)
            print('Total length in bits of sub-packets:', total_length_b, "=", total_length)

            children_packets = in_b[7 + 15 : 7 + 15 + total_length]
            Create_Tree_From_Packets(children_packets, node, begin + 7 + 15)
                
            node.begin = begin + 7 + 15
            node.end = begin + 7 + 15 + total_length
            
            remaining_start = 7 + 15 + total_length
            remaining = in_b[remaining_start : ]            
            
            if len(remaining) and remaining != len(remaining) * '0':
                print('Recurse on remaining 2:', remaining)
                if num_children_packets:
                    Create_Tree_From_Packets(remaining, parent_node, remaining_start, num_children_packets - 1)
                else:
                    Create_Tree_From_Packets(remaining, parent_node, remaining_start)
            else:
                print('No remaining 2!', remaining)
            
        else:
            num_packets_b = in_b[7 : 7 + 11]
            num_packets = int(num_packets_b, 2)
            print('Number of sub-packets immediately contained:', num_packets_b, "=", num_packets)
            
            # Should only do its children
            all_after = in_b[7 + 11 : ]
            Create_Tree_From_Packets(all_after, node, begin + 7 + 11, num_packets)
            
            print('Expected number of sub-packets:', num_packets, 'Actual Number of sub-packets', len(node.children))

            # At this point we know where it ends
            remaining_start = max([x.end for x in node.children]) - begin + 1
            remaining = in_b[remaining_start:]
            print('Remaining', remaining)
            Create_Tree_From_Packets(remaining, parent_node, remaining_start)
             
                
            
            
            #assert len(node.children) == num_packets
            node.begin = begin + 7 + 11
            node.end = max([x.end for x in node.children])
            
            
            
            


In [6]:
in_h = 'D2FE28'
in_h = '38006F45291200'
in_h = 'EE00D40C823060'

in_h = '8A004A801A8002F478'
in_h = '620080001611562C8802118E34'
in_h = 'C0015000016115A2E0802F182340'
in_h = 'A0016C880162017C3686B18A3D4780'
#in_h = data[0]

#in_h = '9C0141080250320F1802104A08'

in_b_raw = bin(int(in_h, 16))[2:]
in_b = in_b_raw
in_b = add_leading_zeros_to_make_multiple_of_four(in_b)
print('Padded:', in_b, 'size:', len(in_b), in_b[8 : 8 + 15])
c = in_b


root = Packet_Node()
Create_Tree_From_Packets(c, root, 0)

print()
root.Print_Tree()

Padded: 101000000000000101101100100010000000000101100010000000010111110000110110100001101011000110001010001111010100011110000000 size: 120 000000010110110


packets_parse, packet: 101000000000000101101100100010000000000101100010000000010111110000110110100001101011000110001010001111010100011110000000 begin 0 num_children_packets: None
Create Node
version 101 = 5
type_id 000 = 0
length_type_ID: 0
Total length in bits of sub-packets: 000000001011011 = 91


packets_parse, packet: 0010001000000000010110001000000001011111000011011010000110101100011000101000111101010001111 begin 22 num_children_packets: None
Create Node
version 001 = 1
type_id 000 = 0
length_type_ID: 1
Number of sub-packets immediately contained: 00000000001 = 1


packets_parse, packet: 0110001000000001011111000011011010000110101100011000101000111101010001111 begin 40 num_children_packets: 1
Create Node
version 011 = 3
type_id 000 = 0
length_type_ID: 1
Number of sub-packets immediately contained: 00000000101 = 5


packets_par

ValueError: invalid literal for int() with base 2: ''

In [365]:
def versions(root, version):
    
    for x in root.children:
        version += [x.version]
        versions(x, version)
            
version = []
versions(root, version)
sum(version)

23

# Part 2

In [378]:
in_h = 'C200B40A82'
in_h = '04005AC33890'
in_h = '880086C3E88112'
in_h = 'CE00C43D881120'
in_h = 'D8005AC2A8F0'
in_h = 'F600BC2D8F'
in_h = '9C005AC2F8F0'
in_h = '9C0141080250320F1802104A08'
#in_h = data[0]

in_b_raw = bin(int(in_h, 16))[2:]
print('Binary Raw', in_b_raw)

in_b = in_b_raw
in_b = add_leading_zeros_to_make_multiple_of_four(in_b)
print('Padded:', in_b, 'size:', len(in_b), in_b[8 : 8 + 15])
c = in_b

root = Packet_Node()
Create_Tree_From_Packets(c, root, 0)

print()
root.Print_Tree()


Binary Raw 10011100000000010100000100001000000000100101000000110010000011110001100000000010000100000100101000001000
Padded: 10011100000000010100000100001000000000100101000000110010000011110001100000000010000100000100101000001000 size: 104 000000010100000


packets_parse, packet: 10011100000000010100000100001000000000100101000000110010000011110001100000000010000100000100101000001000 begin 0 num_children_packets: None
Create Node
version 100 = 4
type_id 111 = 7
length_type_ID: 0
Total length in bits of sub-packets: 000000001010000 = 80


packets_parse, packet: 01000010000000001001010000001100100000111100011000000000100001000001001010000010 begin 22 num_children_packets: None
Create Node
version 010 = 2
type_id 000 = 0
length_type_ID: 1
Number of sub-packets immediately contained: 00000000010 = 2


packets_parse, packet: 01010000001100100000111100011000000000100001000001001010000010 begin 40 num_children_packets: 2
Create Node
version 010 = 2
type_id 100 = 4
Literal Packet
Literal: 0001 =

In [355]:
def Compute_From_Tree(root):
    
    print('\nCompute_From_Tree', [x.type_id for x in root.children])
        
    if root.type_id == 4:
        print('base case', root.literal)
        return root.literal
    
    if root.type_id == 0:
        print('sum')
        res = [Compute_From_Tree(x) for x in root.children]
        print('returning', res)
        return sum(res)
        
    if root.type_id == 1:
        print('prod')
        res = 1
        for x in root.children:
            res *= Compute_From_Tree(x)
        return res

    if root.type_id == 2:
        print('min')
        return min([Compute_From_Tree(x) for x in root.children])
        
    if root.type_id == 3:
        print('max')
        return max([Compute_From_Tree(x) for x in root.children])
    
    if root.type_id == 5:
        print('ge')
        res = [Compute_From_Tree(x) for x in root.children]
        return int(res[0] > res[1])
    
    if root.type_id == 6:
        print('le')
        res = [Compute_From_Tree(x) for x in root.children]
        return int(res[0] < res[1])
    
    if root.type_id == 7:
        print('ee')
        res = [Compute_From_Tree(x) for x in root.children]
        print('res', res)
        return int(res[0] == res[1])
    
    

Compute_From_Tree(root.children[0])


Compute_From_Tree [3, 1, 6, 1]
sum

Compute_From_Tree [4, 4, 4, 4]
max

Compute_From_Tree []
base case 15

Compute_From_Tree []
base case 199071281

Compute_From_Tree []
base case 499

Compute_From_Tree []
base case 190

Compute_From_Tree [4, 5]
prod

Compute_From_Tree []
base case 240

Compute_From_Tree [4, 4]
ge

Compute_From_Tree []
base case 3410

Compute_From_Tree []
base case 201

Compute_From_Tree [0]
le

Compute_From_Tree []
sum
returning []


IndexError: list index out of range